### Appendix C: Compiler Directives

Compiler directives in SystemVerilog are special commands that control the compilation process and provide instructions to the simulator or synthesis tool. They begin with a backtick (`) and are processed before the actual SystemVerilog code compilation.

#### C.1 Text Substitution Directives

##### `define and `undef

The `define directive creates text macros that can be used throughout the design.

```systemverilog
// Basic macro definition
`define WIDTH 32
`define DEPTH 1024

// Macro with parameters
`define MAX(a,b) ((a) > (b) ? (a) : (b))
`define MIN(a,b) ((a) < (b) ? (a) : (b))

// Using macros
module memory_block;
    logic [`WIDTH-1:0] data [`DEPTH-1:0];
    
    initial begin
        int val1 = 10, val2 = 20;
        int maximum = `MAX(val1, val2);
        int minimum = `MIN(val1, val2);
        $display("Max: %0d, Min: %0d", maximum, minimum);
    end
endmodule

// Undefine macro
`undef WIDTH
`undef MAX
```

##### Multi-line Macros

```systemverilog
// Multi-line macro definition
`define CLOCK_GEN(clk_name, period) \
    initial clk_name = 0; \
    always #(period/2) clk_name = ~clk_name;

// Usage
module testbench;
    logic clk;
    `CLOCK_GEN(clk, 10)
    
    initial begin
        #100 $finish;
    end
endmodule
```

##### `ifdef, `ifndef, `else, `elsif, `endif

Conditional compilation directives allow selective compilation of code.

```systemverilog
// Conditional compilation based on defines
`define DEBUG_MODE
`define SYNTHESIS

module processor;
    logic [31:0] instruction;
    
    `ifdef DEBUG_MODE
        // Debug code - only included if DEBUG_MODE is defined
        initial $display("Debug mode enabled");
        
        always @(instruction) begin
            $display("Instruction: %h", instruction);
        end
    `endif
    
    `ifndef SYNTHESIS
        // Simulation-only code
        initial begin
            $dumpfile("processor.vcd");
            $dumpvars(0, processor);
        end
    `else
        // Synthesis-specific code
        // synthesis translate_off
        initial $display("Synthesis mode");
        // synthesis translate_on
    `endif
    
    `ifdef FEATURE_A
        // Feature A implementation
        logic feature_a_enable;
    `elsif FEATURE_B
        // Feature B implementation
        logic feature_b_enable;
    `else
        // Default implementation
        logic default_enable;
    `endif
endmodule
```

#### C.2 File Inclusion Directives

##### `include

The `include directive inserts the contents of another file at the point of inclusion.

```systemverilog
// common_defines.sv
`define BUS_WIDTH 32
`define ADDR_WIDTH 16
`define TRUE  1'b1
`define FALSE 1'b0

// main_module.sv
`include "common_defines.sv"

module cpu;
    logic [`BUS_WIDTH-1:0] data_bus;
    logic [`ADDR_WIDTH-1:0] address_bus;
    logic enable = `TRUE;
endmodule
```

#### C.3 Compiler Control Directives

##### `timescale

Specifies the time unit and precision for simulation.

```systemverilog
// Format: `timescale <time_unit>/<time_precision>
`timescale 1ns/1ps

module timing_example;
    logic clk;
    
    initial begin
        clk = 0;
        #5.5 clk = 1;    // 5.5ns delay
        #10.25 clk = 0;  // 10.25ns delay (rounded to 10.25ns)
    end
endmodule
```

##### `default_nettype

Controls the default net type for implicit declarations.

```systemverilog
// Default is 'wire'
`default_nettype wire

module default_wire_example;
    // implicitly declares 'sig' as wire
    assign sig = 1'b1;
endmodule

// Change default to 'none' to catch undeclared signals
`default_nettype none

module strict_example;
    logic data;  // Must explicitly declare all signals
    // assign undefined_sig = 1'b1;  // This would cause an error
endmodule

// Reset to default
`default_nettype wire
```

##### `celldefine and `endcelldefine

Mark modules as cell definitions for library characterization.

```systemverilog
`celldefine
module and_gate(output y, input a, b);
    assign y = a & b;
endmodule
`endcelldefine
```

#### C.4 Line Control Directives

##### `line

Provides line number and file name information for error reporting.

```systemverilog
`line 100 "original_file.sv" 1
// Compiler will report this as line 100 of original_file.sv
module line_control_example;
    // This will be reported as line 102
    logic [7:0] data;
endmodule
```

##### `__FILE__ and `__LINE__

Predefined macros that expand to current file name and line number.

```systemverilog
module debug_info;
    initial begin
        $display("Current file: %s", `__FILE__);
        $display("Current line: %0d", `__LINE__);
    end
endmodule
```

#### C.5 Pragma Directives

##### `pragma

Provides tool-specific directives without affecting other tools.

```systemverilog
// Synthesis pragmas
`pragma synthesis_off
// This code is ignored during synthesis
initial $display("Simulation only");
`pragma synthesis_on

// Tool-specific pragmas
`pragma translate_off
// Ignored by specific tools
`pragma translate_on
```

#### C.6 Advanced Macro Techniques

##### Stringification and Token Concatenation

```systemverilog
// String conversion
`define STRING(x) `"x`"
`define DISPLAY_VAR(var) $display(`STRING(var) " = %0d", var)

// Token concatenation
`define CONCAT(a,b) a``b
`define REG_DECLARE(name, width) logic [width-1:0] `CONCAT(name,_reg)

module macro_advanced;
    `REG_DECLARE(data, 8)    // Creates: logic [7:0] data_reg
    `REG_DECLARE(addr, 16)   // Creates: logic [15:0] addr_reg
    
    initial begin
        data_reg = 8'hAA;
        `DISPLAY_VAR(data_reg)  // Displays: "data_reg = 170"
    end
endmodule
```

##### Macro Arguments and Default Values

```systemverilog
// Macro with default parameters
`define COUNTER(name, width=8, reset_val=0) \
    logic [width-1:0] name; \
    always_ff @(posedge clk or negedge rst_n) begin \
        if (!rst_n) name <= reset_val; \
        else name <= name + 1; \
    end

module counter_example;
    logic clk, rst_n;
    
    // Use defaults
    `COUNTER(cnt1)              // 8-bit counter, reset to 0
    
    // Override width
    `COUNTER(cnt2, 16)          // 16-bit counter, reset to 0
    
    // Override both
    `COUNTER(cnt3, 12, 'hAAA)   // 12-bit counter, reset to 0xAAA
endmodule
```

#### C.7 Debugging and Documentation Directives

##### `begin_keywords and `end_keywords

Specify SystemVerilog keyword version.

```systemverilog
`begin_keywords "1800-2012"
// Use SystemVerilog-2012 keywords
module sv2012_module;
    // SystemVerilog-2012 specific features
endmodule
`end_keywords
```

##### Custom Assertion Macros

```systemverilog
// Assertion macros for verification
`define ASSERT(condition, message) \
    assert(condition) else $error("ASSERTION FAILED: %s at %s:%0d", \
                                  message, `__FILE__, `__LINE__)

`define ASSERT_CLK(clk, condition, message) \
    assert property (@(posedge clk) condition) \
    else $error("ASSERTION FAILED: %s at %s:%0d", \
                message, `__FILE__, `__LINE__)

module assertion_example;
    logic clk, reset_n, valid, ready;
    
    always_ff @(posedge clk) begin
        `ASSERT(reset_n !== 1'bx, "Reset signal is unknown")
        `ASSERT_CLK(clk, valid |-> ready, "Ready must be high when valid")
    end
endmodule
```

#### C.8 Best Practices for Compiler Directives

##### Coding Standards

```systemverilog
// 1. Use meaningful macro names
`define FIFO_DEPTH_SMALL  16
`define FIFO_DEPTH_MEDIUM 64
`define FIFO_DEPTH_LARGE  256

// 2. Protect against multiple inclusions
`ifndef _COMMON_DEFINES_SV_
`define _COMMON_DEFINES_SV_

// Common definitions here
`define DATA_WIDTH 32

`endif // _COMMON_DEFINES_SV_

// 3. Use consistent naming conventions
`define CFG_ENABLE_FEATURE_A
`define CFG_DISABLE_DEBUG_MODE
`define CFG_SET_TIMEOUT(val) val

// 4. Document complex macros
/**
 * CREATE_FSM - Creates a finite state machine template
 * @param fsm_name: Name of the FSM instance
 * @param state_type: Enumerated type for states
 * @param reset_state: Initial state after reset
 */
`define CREATE_FSM(fsm_name, state_type, reset_state) \
    state_type fsm_name``_current, fsm_name``_next; \
    always_ff @(posedge clk or negedge rst_n) begin \
        if (!rst_n) fsm_name``_current <= reset_state; \
        else fsm_name``_current <= fsm_name``_next; \
    end
```

##### Common Pitfalls and Solutions

```systemverilog
// Pitfall 1: Macro argument side effects
`define BAD_MAX(a,b) ((a) > (b) ? (a) : (b))
// Problem: arguments evaluated multiple times

`define GOOD_MAX(a,b) \
    function automatic int max_func; \
        int temp_a = a, temp_b = b; \
        max_func = (temp_a > temp_b) ? temp_a : temp_b; \
    endfunction : max_func

// Pitfall 2: Missing parentheses in macro expressions
`define BAD_MULTIPLY(a,b) a * b        // Precedence issues
`define GOOD_MULTIPLY(a,b) ((a) * (b)) // Safe precedence

// Pitfall 3: Macro scope pollution
module macro_scope_example;
    `define LOCAL_TEMP 42
    // Use LOCAL_TEMP here
    `undef LOCAL_TEMP  // Clean up after use
endmodule
```

#### C.9 Tool-Specific Directives

##### Synthesis Directives

```systemverilog
// Synthesis control
`ifdef SYNTHESIS
    // synthesis attribute keep_hierarchy "true"
    // synthesis attribute dont_touch "true"
`endif

// Coverage directives
`ifdef COVERAGE
    // coverage off
    initial $display("Coverage collection disabled for this block");
    // coverage on
`endif
```

##### Simulation Directives

```systemverilog
// Simulation-specific optimizations
`ifdef SIMULATION
    `define SIM_DELAY #1
    `define SIM_DISPLAY(msg) $display("[%0t] %s", $time, msg)
`else
    `define SIM_DELAY
    `define SIM_DISPLAY(msg)
`endif
```

#### Summary

Compiler directives are powerful tools that provide fine-grained control over the compilation and simulation process. Key points to remember:

1. **Text Substitution**: Use `define for constants and parameterized macros
2. **Conditional Compilation**: Leverage `ifdef family for configurable designs
3. **File Management**: Use `include for modular code organization
4. **Tool Control**: Apply `timescale, `default_nettype for simulation control
5. **Best Practices**: Follow naming conventions, document complex macros, and avoid common pitfalls

Proper use of compiler directives enhances code reusability, maintainability, and configurability while supporting different design flows and target technologies.